In [1]:
import topmost
from topmost.data import RawDataset
from topmost.preprocessing import Preprocessing
from sklearn.datasets import fetch_20newsgroups

docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']
preprocessing = Preprocessing(vocab_size=10000, stopwords='English')

device = 'cuda' # or 'cpu'
dataset = RawDataset(docs, preprocessing, device=device)

/home/xiaobao/local/miniconda3/envs/topmost/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing texts: 100%|██████████| 18846/18846 [00:01<00:00, 15727.93it/s]
2024-06-19 22:44:06,347 - TopMost - Real vocab size: 10000
2024-06-19 22:44:06,442 - TopMost - Real training size: 18846 	 avg length: 68.727
loading word embeddings: 100%|██████████| 10000/10000 [00:02<00:00, 4655.48it/s]
2024-06-19 22:44:35,116 - TopMost - number of found embeddings: 9869/10000


In [6]:
model = topmost.models.ProdLDA(dataset.vocab_size, num_topics=50)
model = model.to(device)

trainer = topmost.trainers.BasicTrainer(model, dataset, verbose=True)

topic_top_words, doc_topic_dist = trainer.train()

  0%|          | 0/200 [00:00<?, ?it/s]

100%|██████████| 200/200 [00:38<00:00,  5.26it/s]


Topic 0: commandments contexts cor pauls christians argues cites phrases explicitly homosexuality contemporary gentiles wright biblical gentile
Topic 1: sexual homosexual promiscuous marriage church sin partners cramer intercourse homosexuals homosexuality sex lesbian marriages heterosexual
Topic 2: manuals obo printer commodore sale forsale shipping borland compatible panasonic spreadsheet toner cod baud monitor
Topic 3: identity conventions operators abuses liable responsibilities pools columns mechanisms identification oreilly servers privacy originate frontier
Topic 4: casualties mysteries tyre lebanon unified iraqi stars islands temperature bursts dewey lebanese britain iranian magnetic
Topic 5: motherboard slots seagate mhz iisi connector vram card ide simms quadra adapter vlb cache pds
Topic 6: chastity skepticism shameful intellect surrender pcx dortmund dxf gordon rainer download upload bmp autocad pointers
Topic 7: proclaim disobedience prayed rejection persecution christ aut

In [7]:
import torch
from topmost.preprocessing import Preprocessing

new_docs = [
    "This is a document about space, including words like space, satellite, launch, orbit.",
    "This is a document about Microsoft Windows, including words like windows, files, dos."
]

preprocessing = Preprocessing()
new_parsed_docs, new_bow = preprocessing.parse(new_docs, dataset.vocab)
new_theta = trainer.test(torch.as_tensor(new_bow, device=device).float())
print(new_theta.argmax(1))

parsing texts: 100%|██████████| 2/2 [00:00<00:00, 23109.11it/s]

[35  8]
